In [ ]:
%reload_ext autoreload
%autoreload 2

# Useful high-level external modules.
import numpy as np
import pandas as pd
import plotly.express as px

import sys

sys.path.append("../../src")

import oge.load_data as load_data
import oge.data_cleaning as data_cleaning
import oge.filepaths as filepaths

In [ ]:
year = 2022

# load the fuel content data
fuel_heat_content = load_data.load_pudl_table(
    "denorm_generation_fuel_combined_eia923",
    year,
    columns=[
        "plant_id_eia",
        "report_date",
        "energy_source_code",
        "fuel_mmbtu_per_unit",
        "fuel_consumed_mmbtu",
    ],
)
fuel_heat_content = fuel_heat_content.replace([np.inf, -np.inf], np.nan)
fuel_heat_content = fuel_heat_content.dropna()
fuel_heat_content

In [ ]:
# identify which plants have an "OTH" fuel type with non-zero fuel consumption
plants_with_oth = fuel_heat_content[
    fuel_heat_content["energy_source_code"] == "OTH"
].copy()
plants_with_oth = (
    plants_with_oth.groupby("plant_id_eia")[
        ["fuel_mmbtu_per_unit", "fuel_consumed_mmbtu"]
    ]
    .mean()
    .reset_index()
)
plants_with_oth = plants_with_oth[plants_with_oth["fuel_consumed_mmbtu"] != 0]
plants_with_oth

In [ ]:
# load the existing manual fuel map table and check which plants already have been manually matched
# any plants with a missing `updated_energy_source_code` column need to be manually matched
manual_table = pd.read_csv(
    filepaths.reference_table_folder("updated_oth_energy_source_codes.csv")
)
plants_with_oth = plants_with_oth.merge(
    manual_table, how="left", on="plant_id_eia", validate="1:1"
)
plants_with_oth

In [ ]:
# visualize heat content by fuel
# to manually match a fuel type, examine the average `fuel_mmbtu_per_unit` from
# `plants_with_oth` and identify which heat content matches best
px.box(
    fuel_heat_content.sort_values(by="fuel_mmbtu_per_unit"),
    x="energy_source_code",
    y="fuel_mmbtu_per_unit",
    title=f"Range of heat contents for each fuel reported in {year}",
    width=1000,
    height=600,
)